# PyTorch Workflow

Esplorazione di un esempio di PyTorch end-to-end workflow

![workflow](image/01_a_pytorch_workflow.png)


| Topic	| Contents|
|-------|---------|
| 1. Getting data ready | Data can be almost anything but to get started we're going to create a simple straight line| 
| 2. Building a model | Here we'll create a model to learn patterns in the data, we'll also choose a loss function, optimizer and build a training loop.| 
| 3. Fitting the model to data (training) | We've got data and a model, now let's let the model (try to) find patterns in the (training) data.| 
| 4. Making predictions and evaluating a model (inference) | Our model's found patterns in the data, let's compare its findings to the actual (testing) data.| 
| 5. Saving and loading a model | You may want to use your model elsewhere, or come back to it later, here we'll cover that.| 
| 6. Putting it all together | Let's take all of the above and combine it.| 

In [2]:
import torch
from torch import nn # nn contains all of PyTorch's building blocks for neural networks
import matplotlib.pyplot as plt

# Check pytorch version

torch.__version__

'2.7.0'

## 1. Data (Prep and loading)

Nel machine learning i dati possono essere quasi qualsiasi cosa:
* Tabelle excel
* Immagini di ogni tipo
* Video
* Audio
* DNA
* Testo

Il Machine Learning è un gioco composto da due parti:
1. Portare i dati in una forma numerica
2. Costruire un modello che impara pattern da queste rappresentazioni numeriche.

![ML](image/01-machine-learning-a-game-of-two-parts.png)

Useremo la regressione lineare per creare dei dati
$Y_{i}=f(X_i, \beta)+e_i$

* $Y_{i}$ è la variabile dipendente
* $f$ è la funzione
* $X_i$ è la variabile indipendente
* $\beta$ sono i parametri sconosciuti
* $e_i$ errori

Una regressione lineare ha la formula $$Y=a+bX$$ dove $X$ è la va vaiabile indipendente e $Y$ qyella dipendente.
$b$ è la slope (pendenza) e $a$ è la sua intercetta (il valore di $y$ quando $x=0$)